In [ ]:
# %% Doc setup
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
import seaborn as sns
import squarify

global jnt 
global prll 
global chrd

sns.set(style="dark",
        palette="PuBuGn_d",
        font="DejaVu Sans")

import holoviews as hv
from holoviews import opts, dim
from bokeh.sampledata.les_mis import data
from IPython.display import display_html
from holoviews import layout

hv.extension("bokeh")

In [ ]:
# %% Read in movies data
global movies
movies = pd.read_csv("movies.csv")
movies["Vote"]=movies["Vote"].astype("float64")
movies["Runtime"]=movies["Runtime"].astype("float64")
movies[["Genre1","Genre2","Genre3"]]=movies.Genre.str.split(", ",expand=True)
movies["Revenue/Min"]=movies["Revenue"]/movies["Runtime"]
movies["Meta:Score"]=movies["Metascore"]/movies["Score"]*10
movies["Score Decile"]=movies["Score"].apply(np.floor)
movies["Metascore Decile"]=(movies["Metascore"]/10).apply(np.floor)
movies["Decade"]=((movies["Year"]/10).apply(np.floor))*10

#movies.to_csv("movies_out.csv", index=False)

In [ ]:
# %% Useful lists
all_genres = movies["Genre1"].append(movies["Genre2"]).append(movies["Genre3"]).dropna().unique().tolist()
all_directors = movies["Director"].dropna().unique().tolist()

In [ ]:
# %% Take a look
movies.info()
movies.head()
movies.describe()

In [ ]:
# %%
sns.pairplot(movies)

In [ ]:
# %% score vs metascore
jnt = sns.jointplot(x="Score",
              y="Metascore",
              data=movies,
              kind="reg",
              dropna=True)# "scatter" | "reg" | "resid" | "kde" | "hex"

#jnt.savefig("jointplot.png")

In [ ]:
# %% chord- genres
movies_genres12 = pd.crosstab(movies.Genre1, movies.Genre2)
movies_genres13 = pd.crosstab(movies.Genre1, movies.Genre3)

movies_chord12 = hv.Dataset((list(movies_genres12.columns),
                           list(movies_genres12.index),
                           movies_genres12),
                          ['source', 'target'], 'value_12').dframe()

movies_chord13 = hv.Dataset((list(movies_genres13.columns),
                           list(movies_genres13.index),
                           movies_genres13),
                          ['source', 'target'], 'value_13').dframe()

movies_chord = pd.DataFrame.merge(movies_chord12,movies_chord13, how="outer")
movies_chord["value"]=movies_chord.value_12+movies_chord.value_13.fillna(0).astype(int)
movies_chord.drop(["value_12","value_13"], 1, inplace=True)

chrd = hv.Chord(movies_chord).opts(
    title="Genre 1 to Genres 2 and 3",
    node_color='index',
    edge_color='source',
    label_index='index', 
    cmap='Category20b',
    edge_cmap='Category20b',
    width=1000,
    height=1000)

#hv.save(chrd,"chord.html", backend="bokeh")

In [ ]:
# %% score decile vs other quant vars
movies_parallel=movies[["Metascore Decile","Score","Metascore","Vote","Runtime","Revenue","Revenue/Min","Meta:Score"]]
movies_parallel_norm=(movies_parallel.drop("Metascore Decile", axis=1)-movies_parallel.drop("Metascore Decile", axis=1).mean())/movies_parallel.drop("Metascore Decile", axis=1).std()
movies_parallel_norm["Metascore Decile"]=movies["Metascore Decile"]
#parallel_coordinates(movies_parallel_norm, "Metascore Decile")
#plt.show()

movies_parallel_desc = movies_parallel.groupby("Metascore Decile").describe()[zip(movies_parallel.columns[1:],np.repeat("mean",8))]
movies_parallel_desc["Metascore Decile"]=movies_parallel_desc.index
movies_parallel_desc_norm=(movies_parallel_desc.drop("Metascore Decile", axis=1)-movies_parallel_desc.drop("Metascore Decile", axis=1).mean())/movies_parallel_desc.drop("Metascore Decile", axis=1).std()
movies_parallel_desc_norm["Metascore Decile"]=movies_parallel_desc_norm.index

prll = parallel_coordinates(movies_parallel_desc_norm, "Metascore Decile")
plt.show()